In [1]:
import pandas as pd
import numpy as np
import configparser
import pymysql
import requests
import re
from tqdm import tqdm
import configparser
import os

In [2]:
config=configparser.ConfigParser()
config.read('/home/ubuntu/infoedge/llama-recipes/examples/configs/config.ini')

['/home/ubuntu/infoedge/llama-recipes/examples/configs/config.ini']

In [3]:
mydb= pymysql.connect(
host=config['iimjobs_db']['hostname'],
user=config['iimjobs_db']['username'],
password=config['iimjobs_db']['password'],
db=config['iimjobs_db']['db_name'])

mycursor = mydb.cursor()

In [4]:

query = "select id from profile_info where created_at>'2024-03-16' and resume_first_uploaded > '2024-03-16'"

mycursor.execute(query)
result = mycursor.fetchall()

In [5]:
active_users = pd.DataFrame(result,columns=['userid'])

In [6]:
active_users.drop_duplicates(inplace=True)

In [7]:
active_users

,userid
0,2826298
1,2826299
2,2826304
3,2826305
4,2826308
...,...
739,2827743
740,2827745
741,2827746
742,2827748


In [9]:
ulist = tuple(active_users['userid'].tolist())

In [10]:
from user_info import get_user_info

In [12]:
r_text = {}
for uid in tqdm(ulist):
    response = get_user_info(uid)
    try:
        r_text[uid] = response['resume'][0]
    except:
        pass

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 744/744 [00:05<00:00, 145.75it/s]


In [14]:
import json
with open('../custom_data/mistral/new_users_march24.json','w') as f:
    json.dump(r_text,f)